In [1]:
# Bigger dataset available here http://s3.amazonaws.com/hr-testcases-us-east-1/329/assets/bigger_data_set.txt

The first line contains three space separated numbers m k d.
* m - the amount of money you could spend that day.
* k - the number of different stocks available for buying or selling.
* d - the number of remaining days for trading stocks.

k lines follow, each in the following format: name owned prices
* name - the name of the stock (a string).
* owned - the number of shares you own of that stock.
* prices - 5 space separated numbers representing the stock's price for the last 5 days. These are ordered from oldest to newest, so the last number is the current stock price.

Sample input

90 2 400

iStreet 10 4.54 5.53 6.56 5.54 7.60

HR 5 30.54 27.53 24.42 20.11 17.50

In [2]:
import pandas as pd

In [3]:
# Take first line of input, it contains useful info.
info = input().split()
m_today = float(info[0])  # m - the amount of money you could spend that day.
k = int(info[1])  # k - the number of different stocks available for buying or selling.
d = int(info[2])  # d - the number of remaining days for trading stocks.

90 2 400


In [4]:
inpt = [input().split() for line in range(k)]

iStreet 10 4.54 5.53 6.56 5.54 7.60
HR 5 30.54 27.53 24.42 20.11 17.50


In [5]:
for i in range(k):
    for j in range(2,7):
        inpt[i][j] = float(inpt[i][j])
    
    inpt[i][1] = int(inpt[i][1])

        
inpt

[['iStreet', 10, 4.54, 5.53, 6.56, 5.54, 7.6],
 ['HR', 5, 30.54, 27.53, 24.42, 20.11, 17.5]]

In [6]:
# Plot the price fluctuation for the first stock.

pd.Series(inpt[0][2:]).plot.line()

In [7]:
def Slope_calc(lst):
    import numpy as np
    
    name = lst[0]
    owned = lst[1]
    prices = lst[2:]
    
    # create sequential array representing days
    days = np.arange(5)
    # Use numpy method polyfit() with degree '1' to get the slope value
    fit = np.polyfit(days, prices, 1)
    
    
    return([name, owned, prices[-1], fit[0]])

In [8]:
# Creating dataframe for decision making                  
tmp_lst = []
for i in range(len(inpt)):
    tmp_lst.append(Slope_calc(inpt[i]))

df = pd.DataFrame(tmp_lst, columns=['name', 'owned', 'today price', 'slope'])

# Adding ACTION and QTY columns:
df['ACTION'] = ''
df['QTY'] = 0


# Sort by descending slope meaning the more valueable stock to buy.
df.sort_values(by=['slope'], ascending = False)

,name,owned,today price,slope,ACTION,QTY
0,iStreet,10,7.6,0.613,,0
1,HR,5,17.5,-3.350,,0


In [9]:
def Actions(df, m_today):
    for index, row in df.iterrows():
        if row['slope'] > 0:
            if m_today//row['today price'] > 0:
                df.at[index,'ACTION'] = 'BUY'
                df.at[index,'QTY'] = m_today//row['today price']
                m_today -=  row['today price'] * (m_today//row['today price'])
                m_today = round(m_today,2)
        if row['slope'] < 0:
            if row['owned'] > 0:
                df.at[index,'ACTION'] = 'SELL'
                df.at[index,'QTY'] = int(row['owned'])

In [10]:
Actions(df, 90)

In [11]:
df

,name,owned,today price,slope,ACTION,QTY
0,iStreet,10,7.6,0.613,BUY,11
1,HR,5,17.5,-3.350,SELL,5


In [12]:
# Filter rows where the action was taken (meaning QTY is more than zero)
df_actionable = df[df.QTY>0]

# Print transaction
print(len(df_actionable))
for index, row in df_actionable.iterrows():
    print(row['name'], row['ACTION'], row['QTY'])

2
iStreet BUY 11
HR SELL 5


# HackerRank upload

In [13]:
import pandas as pd

# Take first line of input, it contains useful info.
info = input().split()
m_today = float(info[0])  # m - the amount of money you could spend that day.
k = int(info[1])  # k - the number of different stocks available for buying or selling.
d = int(info[2])  # d - the number of remaining days for trading stocks.

# Taking the rest of input
inpt = [input().split() for line in range(k)]

# Formatting the numbers
for i in range(k):
    for j in range(2,7):
        inpt[i][j] = float(inpt[i][j])
    
    inpt[i][1] = int(inpt[i][1])

# Function calculating the slope of prices change.

def Slope_calc(lst):
    import numpy as np
    
    name = lst[0]
    owned = lst[1]
    prices = lst[2:]
    
    # create sequential array representing days
    days = np.arange(5)
    # Use numpy method polyfit() with degree '1' to get the slope value
    fit = np.polyfit(days, prices, 1)
    
    
    return([name, owned, prices[-1], fit[0]])

# Creating dataframe for decision making                  

tmp_lst = []
for i in range(len(inpt)):
    tmp_lst.append(Slope_calc(inpt[i]))

df = pd.DataFrame(tmp_lst, columns=['name', 'owned', 'today price', 'slope'])

# Adding ACTION and QTY columns:
df['ACTION'] = ''
df['QTY'] = 0

# Sort by descending slope meaning the more valueable stock to buy.
df.sort_values(by=['slope'], ascending = False)

# Function to make BUY/SELL decisions.
# When slope is positive - BUY, when negative - SELL.

def Actions(df, m_today):
    for index, row in df.iterrows():
        if row['slope'] > 0:
            if m_today//row['today price'] != 0:
                df.at[index,'ACTION'] = 'BUY'
                df.at[index,'QTY'] = m_today//row['today price']
                m_today -=  row['today price'] * (m_today//row['today price'])
                m_today = round(m_today,2)
        if row['slope'] < 0:
            if row['owned'] > 0:
                df.at[index,'ACTION'] = 'SELL'
                df.at[index,'QTY'] = int(row['owned'])

Actions(df, m_today)

# Filter rows where the action was taken (meaning QTY is more than zero)
df_actionable = df[df.QTY>0]

# Print transaction
if len(df_actionable)==0:
    print(0)
else:
    print(len(df_actionable))
    for index, row in df_actionable.iterrows():
        print(row['name'], row['ACTION'], row['QTY'])
    

90 2 400
iStreet 10 4.54 5.53 6.56 5.54 7.60
HR 5 30.54 27.53 24.42 20.11 17.50
2
iStreet BUY 11
HR SELL 5


90 2 400
iStreet 10 4.54 5.53 6.56 5.54 7.60
HR 5 30.54 27.53 24.42 20.11 17.50